Импортируем numpy для векторных операций.

In [1]:
import numpy as np

Напишем реализацию метода Недлера - Мида.

In [2]:
def find_min_nelder_mead(func, p1, p2, p3, alpha=1, beta=0.5, gamma=2, tolerance=1e-10, tolerance_iterations=100):
    prev_xl = None
    tol_it = 0
    while True:
        # 1
        psorted = [[p1, func(p1)], [p2, func(p2)], [p3, func(p3)]]
        # 2
        psorted.sort(key=lambda x: x[1])
        xl = psorted[0][0]
        xg = psorted[1][0]
        xh = psorted[2][0]
        # 3
        xc = (xg + xl) / 2
        # 4
        xr = (1 + alpha) * xc - alpha * xh
        # 5
        if (func(xr) < func(xl)):
            xe = (1 - gamma) * xc + gamma * xr
            if (func(xe) < func(xr)):
                xh = xe
            else:
                xh = xr
        elif (func(xr) < func(xg)):
            xh = xr
        else:
            if (func(xr) < func(xh)):
                t = xr
                xr = xh
                xh = t

            # 6
            xs = beta * xh + (1 - beta) * xc
            # 7
            if (func(xs) < func(xh)):
                xh = xs
            # 8
            else:
                xh = xl + (xh - xl) / 2
                xg = xl + (xg - xl) / 2

        # 9
        if ((not (prev_xl is None)) and np.sum(np.abs(prev_xl - xl)) < tolerance):
            tol_it += 1
            if (tol_it > tolerance_iterations):
                return xl
        else:
            tol_it = 0

        prev_xl = xl

        p1 = xh
        p2 = xg
        p3 = xl

Определим функцию из условия.

In [3]:
def f(point):
    x, y = point
    return np.sin(y) * np.exp((1 - np.cos(x)) ** 2) +\
        np.cos(x) * np.exp((1 - np.sin(y)) ** 2) + (x - y) ** 2

Покажем, что при разных значениях начальных треугольников (точки p1, p2, p3) метод находит разные минимумы.

In [4]:
p1 = np.array([0, 0])
p2 = np.array([0, 1])
p3 = np.array([1, 0])
print(find_min_nelder_mead(f, p1, p2, p3))

p1 = np.array([-6, -6])
p2 = np.array([-6, -8])
p3 = np.array([-8, -6])
print(find_min_nelder_mead(f, p1, p2, p3))

p1 = np.array([6, 6])
p2 = np.array([6, 8])
p3 = np.array([8, 8])
print(find_min_nelder_mead(f, p1, p2, p3))


[0.9055187  0.66527763]
[-5.37766661 -5.61790768]
[9.3906441  4.74652283]


Покажем теперь, что параметры $\alpha,$ $\beta$ и $\gamma$ также влияют на точку минимума, к которой сходится метод, причем при некоторых параметрах метод сходится к одним и тем же минимумам, но с разной точностью.

In [5]:
p1 = np.array([10, 0])
p2 = np.array([0, 10])
p3 = np.array([10, 10])

alpha = 1
beta = 0.5
gamma = 2
print(find_min_nelder_mead(f, p1, p2, p3, alpha, beta, gamma))

alpha = 0.1
beta = 0.9
gamma = 2
print(find_min_nelder_mead(f, p1, p2, p3, alpha, beta, gamma))

alpha = 3
beta = 0.1
gamma = 1.5
print(find_min_nelder_mead(f, p1, p2, p3, alpha, beta, gamma))


[9.39064411 4.74652284]
[ 9.99516583 10.01992271]
[ 9.41107789 10.96051299]


Мы видим, что при первом наборе параметров метод сходится к минимуму $(9.39064411, 4.74652284)$. Но при двух других наборах метод сходится к другим минимумам, но если анализировать функцию, то можно увидеть, что эти минимумы одни и те же, но найдены с разной точностью.